In [152]:
import pandas as pd
import re
import nltk
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
import sys
import numpy
from sklearn.model_selection import train_test_split
import tensorflow as tf
import datetime
%load_ext tensorboard
numpy.set_printoptions(threshold=sys.maxsize)
nltk.download('stopwords')
pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Logan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
amazon = pd.read_csv("D:\MS DA\D213\Task 2\sentiment labelled sentences\\amazon_cells_labelled.txt", sep = "\t", header=None, names=["text", "score"]);\
amazon.shape

(1000, 2)

In [41]:
imdb = pd.read_csv("D:\MS DA\D213\Task 2\sentiment labelled sentences\\imdb_labelled.csv", header=None, names=["text", "score"]);\
imdb.head

<bound method NDFrame.head of                                                   text  score
0    A very, very, very slow-moving, aimless movie ...    0.0
1    Not sure who was more lost - the flat characte...    0.0
2    Attempting artiness with black & white and cle...    0.0
3         Very little music or anything to speak of.      0.0
4    The best scene in the movie was when Gerardo i...    1.0
..                                                 ...    ...
995  I just got bored watching Jessice Lange take h...    0.0
996  Unfortunately, any virtue in this film's produ...    0.0
997                   In a word, it is embarrassing.      0.0
998                               Exceptionally bad!      0.0
999  All in all its an insult to one's intelligence...    0.0

[1000 rows x 2 columns]>

In [42]:
yelp = pd.read_csv("D:\MS DA\D213\Task 2\sentiment labelled sentences\\yelp_labelled.csv", header=None, names=["text", "score"]);\
yelp.head

<bound method NDFrame.head of                                                   text  score
0                             Wow... Loved this place.      1
1                                   Crust is not good.      0
2            Not tasty and the texture was just nasty.      0
3    Stopped by during the late May bank holiday of...      1
4    The selection on the menu was great and so wer...      1
..                                                 ...    ...
995  I think food should have flavor and texture an...      0
996                           Appetite instantly gone.      0
997  Overall I was not impressed and would not go b...      0
998  The whole experience was underwhelming, and I ...      0
999  Then, as if I hadn't wasted enough of my life ...      0

[1000 rows x 2 columns]>

In [68]:
reviews = pd.concat([amazon, imdb, yelp])
reviews.dropna(inplace=True)
reviews.head

reviews['score'] = reviews['score'].astype(int)
reviews.head
reviews.dtypes

text     object
score     int32
dtype: object

In [70]:
reviews['text'] = reviews['text'].str.replace("'", "")
reviews['text'] = reviews['text'].str.replace(r'[^\w\s]+', ' ')
reviews['text'] = reviews['text'].str.replace('\d+', '')
reviews['text'] = reviews['text'].str.replace('  ', ' ')
reviews['text'] = reviews['text'].str.lower()
display(reviews.head)

C:\Users\Logan\AppData\Local\Temp\ipykernel_11400\2193798075.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  reviews['text'] = reviews['text'].str.replace(r'[^\w\s]+', ' ')
C:\Users\Logan\AppData\Local\Temp\ipykernel_11400\2193798075.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  reviews['text'] = reviews['text'].str.replace('\d+', '')


<bound method NDFrame.head of                                                                                                                                     text  \
0                                                     so there is no way for me to plug it in here in the us unless i go by a converter    
1                                                                                                             good case excellent value    
2                                                                                                                 great for the jawbone    
3                                                            tied to charger for conversations lasting more than minutes major problems    
4                                                                                                                      the mic is great    
..                                                                                                                                

In [74]:
def tokenize(text):
    words = re.split('\W+', text)
    return words

reviews['token_text'] = reviews['text'].apply(lambda x: tokenize(x))
reviews.head()

,text,score,token_text
0,so there is no way for me to plug it in here in the us unless i go by a converter,0,"[so, there, is, no, way, for, me, to, plug, it, in, here, in, the, us, unless, i, go, by, a, converter, ]"
1,good case excellent value,1,"[good, case, excellent, value, ]"
2,great for the jawbone,1,"[great, for, the, jawbone, ]"
3,tied to charger for conversations lasting more than minutes major problems,0,"[tied, to, charger, for, conversations, lasting, more, than, minutes, major, problems, ]"
4,the mic is great,1,"[the, mic, is, great, ]"


In [85]:
stopword = nltk.corpus.stopwords.words('english')

def stopwords(token_text):
    text = [word for word in token_text if word not in stopword]
    return text

reviews['nostop_text'] = reviews['token_text'].apply(lambda x: stopwords(x))
reviews.head()

,text,score,token_text,nostop_text
0,so there is no way for me to plug it in here in the us unless i go by a converter,0,"[so, there, is, no, way, for, me, to, plug, it, in, here, in, the, us, unless, i, go, by, a, converter, ]","[way, plug, us, unless, go, converter, ]"
1,good case excellent value,1,"[good, case, excellent, value, ]","[good, case, excellent, value, ]"
2,great for the jawbone,1,"[great, for, the, jawbone, ]","[great, jawbone, ]"
3,tied to charger for conversations lasting more than minutes major problems,0,"[tied, to, charger, for, conversations, lasting, more, than, minutes, major, problems, ]","[tied, charger, conversations, lasting, minutes, major, problems, ]"
4,the mic is great,1,"[the, mic, is, great, ]","[mic, great, ]"


In [88]:
max_words = max([len(t_line) for t_line in reviews['nostop_text']])
max_words

45

In [91]:
reviews['final'] = reviews['nostop_text'].apply(lambda x: ' '.join(x))
reviews.head()

,text,score,token_text,nostop_text,final
0,so there is no way for me to plug it in here in the us unless i go by a converter,0,"[so, there, is, no, way, for, me, to, plug, it, in, here, in, the, us, unless, i, go, by, a, converter, ]","[way, plug, us, unless, go, converter, ]",way plug us unless go converter
1,good case excellent value,1,"[good, case, excellent, value, ]","[good, case, excellent, value, ]",good case excellent value
2,great for the jawbone,1,"[great, for, the, jawbone, ]","[great, jawbone, ]",great jawbone
3,tied to charger for conversations lasting more than minutes major problems,0,"[tied, to, charger, for, conversations, lasting, more, than, minutes, major, problems, ]","[tied, charger, conversations, lasting, minutes, major, problems, ]",tied charger conversations lasting minutes major problems
4,the mic is great,1,"[the, mic, is, great, ]","[mic, great, ]",mic great


In [131]:
tfidf = TfidfVectorizer()
all_words = tfidf.fit_transform(reviews['final'])
print(all_words)
print(all_words.shape)
print(tfidf.get_feature_names_out())

  (0, 923)	0.5344060724219368
  (0, 1885)	0.31058252544651554
  (0, 4647)	0.44527538534051525
  (0, 4687)	0.3694947051312881
  (0, 3260)	0.4140600486719989
  (0, 4833)	0.3342250960495995
  (1, 4708)	0.6340788281411345
  (1, 1506)	0.4707096091113773
  (1, 622)	0.5130637026931167
  (1, 1897)	0.3363660811856765
  (2, 2337)	0.8994525647615415
  (2, 1926)	0.43701840206550224
  (3, 3355)	0.3279329838852072
  (3, 2639)	0.42009913582909286
  (3, 2779)	0.29601151425839517
  (3, 2452)	0.404813459898733
  (3, 922)	0.404813459898733
  (3, 677)	0.32489593335506933
  (3, 4445)	0.4416430860335276
  (4, 2755)	0.8942369972581401
  (4, 1926)	0.44759378093841423
  (5, 4776)	0.3357207918384368
  (5, 1081)	0.34939780890724126
  (5, 3647)	0.28560254509472904
  (5, 2535)	0.32489657710785297
  :	:
  (2992, 4947)	0.3866700676920576
  (2992, 2199)	0.5092990475248643
  (2992, 1885)	0.40940592186906716
  (2993, 2929)	0.4317061729061377
  (2993, 4298)	0.3306386887412905
  (2993, 4621)	0.41064695883450425
  (2993, 

In [118]:
features = pd.DataFrame(all_words.toarray())
features.head

<bound method NDFrame.head of       0     1     2     3     4     5     6     7     8     9     ...  4986  \
0      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
1      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
2      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
3      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
4      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
2990   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
2991   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
2992   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
2993   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
2994   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   

      4987  4988  498

In [121]:
features.to_csv("D:\\MS DA\\D213\\Task 2\\reviews_clean.csv")

In [129]:
x = features
y = reviews['score']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=22)
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.15, random_state=22)

print("Training size: ", x_train.shape)
print("Test size: ", x_test.shape)
print("Validation size: ", x_val.shape)

Training size:  (2545, 4996)
Test size:  (599, 4996)
Validation size:  (450, 4996)


In [132]:
x_train.to_csv("D:\\MS DA\\D213\\Task 2\\x_train.csv")
y_train.to_csv("D:\\MS DA\\D213\\Task 2\\y_train.csv")
x_test.to_csv("D:\\MS DA\\D213\\Task 2\\x_test.csv")
y_test.to_csv("D:\\MS DA\\D213\\Task 2\\y_test.csv")
x_val.to_csv("D:\\MS DA\\D213\\Task 2\\x_val.csv")
y_val.to_csv("D:\\MS DA\\D213\\Task 2\\y_val.csv")

In [136]:
tensor_dataset = tf.data.Dataset.from_tensor_slices((features.values, y.values))
train_tf_dataset = tensor_dataset.shuffle(len(reviews)).batch(1)

In [146]:
def compiled_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    
    model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
    return model

In [147]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"), histogram_freq=1)

In [155]:
model = compiled_model()
history = model.fit(train_tf_dataset, epochs=15, callbacks=[tensorboard_callback])
model.summary()

Epoch 1/15
2995/2995 [==============================] - 3s 801us/step - loss: 0.1851 - accuracy: 0.7285
Epoch 2/15
2995/2995 [==============================] - 2s 802us/step - loss: 0.0623 - accuracy: 0.9229
Epoch 3/15
2995/2995 [==============================] - 2s 796us/step - loss: 0.0283 - accuracy: 0.9699
Epoch 4/15
2995/2995 [==============================] - 2s 806us/step - loss: 0.0188 - accuracy: 0.9763
Epoch 5/15
2995/2995 [==============================] - 3s 855us/step - loss: 0.0137 - accuracy: 0.9860
Epoch 6/15
2995/2995 [==============================] - 2s 812us/step - loss: 0.0125 - accuracy: 0.9870
Epoch 7/15
2995/2995 [==============================] - 2s 796us/step - loss: 0.0113 - accuracy: 0.9886
Epoch 8/15
2995/2995 [==============================] - 2s 797us/step - loss: 0.0100 - accuracy: 0.9893
Epoch 9/15
2995/2995 [==============================] - 2s 799us/step - loss: 0.0095 - accuracy: 0.9903
Epoch 10/15
2995/2995 [==============================] - 2s 811u

In [158]:
direct = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
%tensorboard --direct logs/fit/;

ERROR: Failed to launch TensorBoard (exited with 2).
Contents of stderr:
usage: tensorboard [-h] [--helpfull] [--logdir PATH] [--logdir_spec PATH_SPEC]
                   [--host ADDR] [--bind_all] [--port PORT]
                   [--reuse_port BOOL] [--load_fast {false,auto,true}]
                   [--extra_data_server_flags EXTRA_DATA_SERVER_FLAGS]
                   [--grpc_creds_type {local,ssl,ssl_dev}]
                   [--grpc_data_provider PORT] [--purge_orphaned_data BOOL]
                   [--db URI] [--db_import] [--inspect] [--version_tb]
                   [--tag TAG] [--event_file PATH] [--path_prefix PATH]
                   [--window_title TEXT] [--max_reload_threads COUNT]
                   [--reload_interval SECONDS] [--reload_task TYPE]
                   [--reload_multifile BOOL]
                   [--reload_multifile_inactive_secs SECONDS]
                   [--generic_data TYPE]
                   [--samples_per_plugin SAMPLES_PER_PLUGIN]
                   [-

In [159]:
model = compiled_model()
history = model.fit(train_tf_dataset, epochs=10, callbacks=[tensorboard_callback])
model.summary()

Epoch 1/10
2995/2995 [==============================] - 3s 789us/step - loss: 0.1817 - accuracy: 0.7432
Epoch 2/10
2995/2995 [==============================] - 2s 789us/step - loss: 0.0639 - accuracy: 0.9242
Epoch 3/10
2995/2995 [==============================] - 2s 785us/step - loss: 0.0287 - accuracy: 0.9676
Epoch 4/10
2995/2995 [==============================] - 2s 792us/step - loss: 0.0178 - accuracy: 0.9803
Epoch 5/10
2995/2995 [==============================] - 2s 800us/step - loss: 0.0130 - accuracy: 0.9866
Epoch 6/10
2995/2995 [==============================] - 3s 828us/step - loss: 0.0118 - accuracy: 0.9876
Epoch 7/10
2995/2995 [==============================] - 2s 795us/step - loss: 0.0109 - accuracy: 0.9876
Epoch 8/10
2995/2995 [==============================] - 2s 792us/step - loss: 0.0096 - accuracy: 0.9886
Epoch 9/10
2995/2995 [==============================] - 2s 797us/step - loss: 0.0081 - accuracy: 0.9907
Epoch 10/10
2995/2995 [==============================] - 2s 783u